In [1]:
#Import packages
import tensorflow as tf
import matplotlib.pyplot as plt
import time

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)


KeyboardInterrupt: 

In [ ]:
# Parameters
with tf.name_scope("parameters"):
    learning_rate = 0.001
    training_epochs = 20
    batch_size = 100
    display_epoch = 1
    logs_path = '/tmp/tensorflow_logs/example/'
    #M=28*28 #dimension
    #K=10# of class

    
# tf Graph Input
# mnist data image of shape 28*28 =784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
t = tf.placeholder(tf.float32, [None, 10], name='LabelData')

#print image 
with tf.name_scope("Input_shape"):
    image=tf.reshape(x,[-1,28,28,1])
    tf.summary.image('Input',image,10)
    
# dropout (keep_prob) rate  0.7~0.5 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)
    
# Set model weights and bias 
# w = tf.Variable(tf.zeros([784, 10]), name='Weights')
# b = tf.Variable(tf.zeros([10]), name='Bias')
    

# For printing summary of variables : mean, max, min, histogram 
def variable_summaries(variable):
     with tf.name_scope('Summaries'):
            tf.summary.scalar('Mean', tf.reduce_mean(variable))
            tf.summary.scalar('Max', tf.reduce_max(variable))
            tf.summary.scalar('Min', tf.reduce_min(variable))
            tf.summary.histogram('Histogram', variable)    

#For changing y 
def layer(input_data, input_dim, output_dim, layer_name, hidden, act=tf.nn.relu): #act : activation ftn
    with tf.name_scope(layer_name):
        with tf.name_scope('Weights'):
            w=tf.get_variable(layer_name +"W",[input_dim, output_dim], initializer=tf.contrib.layers.xavier_initializer())
            variable_summaries(w)
        with tf.name_scope('Bias'):
            b = tf.Variable(tf.random_normal([output_dim]))
            variable_summaries(b)
        with tf.name_scope('Y'):
            if(hidden == 1):
                preactivate = tf.matmul(input_data, w) + b
                y= act(preactivate)
                y= tf.nn.dropout(y,keep_prob)
                tf.summary.histogram('y', y)
            else : 
                y = tf.matmul(input_data, w) + b
                tf.summary.histogram('final_y', y)
        return y

hidden1 = layer(x, 784, 512, 'layer1' ,1)
hidden2 = layer(hidden1 , 512, 512, 'layer2',1)
hidden3 = layer(hidden2, 512, 512, 'layer3',1)
hidden4 = layer(hidden3, 512, 512, 'layer4',1)
y = layer(hidden3, 512,10, 'layer5',0)

with tf.name_scope('Cross_entropy'):
    cross_entropy= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=y, labels=t))   # Minimize error using cross entropy
    tf.summary.scalar('Cross_entropy',cross_entropy)      
           
with tf.name_scope('Train_step'): 
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy) 
    
with tf.name_scope('Accuracy'):
    correct_prediction= tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))
    accuracy= tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("Accuracy",accuracy)
    
##########################################################################################
# Start training 
with tf.Session() as sess:
    # Initialize the variables 
    init = tf.global_variables_initializer()
    # Merge all summaries into a single op
    merged = tf.summary.merge_all()
    # Writer for two options : for training data and test data
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Run the initializer
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        
        avg_cross_entropy = 0.
        n_batches = int(mnist.train.num_examples/batch_size)
        
        for i in range(n_batches):
            batch_xs, batch_ts = mnist.train.next_batch(batch_size)
            #training data accuracy
            
            _, c, summary = sess.run([train_step, cross_entropy, merged],
                                     feed_dict={x: batch_xs, t: batch_ts, keep_prob: 0.7})

            # Write logs at every iteration
            writer.add_summary(summary, epoch * n_batches + i)
            # Compute average loss
            avg_cross_entropy += c /n_batches

        # Display logs per epoch step
        if (epoch+1) % display_epoch== 0:
            print("Epoch", '%d/20' % (epoch+1), "cross_entropy=", "{:.9f}".format(avg_cross_entropy))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy
    print("Test Accuracy", sess.run(accuracy,feed_dict={x: mnist.test.images, t: mnist.test.labels ,keep_prob:1}))
    print("Training Accuracy", sess.run(accuracy,feed_dict={x: mnist.train.images, t: mnist.train.labels, keep_prob:1}))


    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs")
        

Epoch 1/20 cross_entropy= 0.386491062
